In [1]:
import pandas as pd
import numpy as np
from numpy import random
from datetime import datetime
from datetime import timedelta
from dateutil.rrule import DAILY, rrule, MO, TU, WE, TH, FR, SA, SU
import calendar
from chinesename import ChineseName
import names

In [2]:
# DECLARE START AND END DATES
start = '2013-03-01'
end = '2015-07-31'
# GET NUMBER OF WORKDAYS
weekmask = 'Wed Thu Fri Sat Sun'
customworkweek = pd.offsets.CustomBusinessDay(weekmask=weekmask)
workdates = pd.bdate_range(start = start, end = end, freq = customworkweek).to_pydatetime().tolist()
# GET NUMBER OF WEEKS AND MONTHS 
x = pd.to_datetime(end) - pd.to_datetime(start)
workweeks_int = int(x / np.timedelta64(1, 'W'))
workmonths_int = int(x / np.timedelta64(1, 'M'))

In [3]:
# DEFINE GLOBAL VARIABLES
student_count = 500
classes_count = 27
participation_count = 300000
test_score_max = 50
hw_score_max = 10
hw_count = len(workdates)*classes_count

In [4]:
# GET CLASS DATES FOR EACH CLASS
wed = pd.offsets.CustomBusinessDay(weekmask='Wed')
thu = pd.offsets.CustomBusinessDay(weekmask='Thu')
fri = pd.offsets.CustomBusinessDay(weekmask='Fri')
sat = pd.offsets.CustomBusinessDay(weekmask='Sat')
sun = pd.offsets.CustomBusinessDay(weekmask='Sun')

wed_classes = pd.bdate_range(start = start, end = end, freq = wed).to_pydatetime().tolist()
thu_classes = pd.bdate_range(start = start, end = end, freq = thu).to_pydatetime().tolist()
fri_classes = pd.bdate_range(start = start, end = end, freq = fri).to_pydatetime().tolist()
sat_classes = pd.bdate_range(start = start, end = end, freq = sat).to_pydatetime().tolist()
sun_classes = pd.bdate_range(start = start, end = end, freq = sun).to_pydatetime().tolist()

## STUDENTS.CSV

In [5]:
# GENERATE A LIST OF 500 CHINESE NAMES
names_local = ChineseName().getNames(student_count)
# GENERATE A LIST OF ENGLISH FIRST NAMES
names_english = []
for _ in range(student_count):
    x = names.get_first_name()
    names_english.append(x)
student_age = random.randint(4, 13, size = (student_count))
student_sex = random.choice(["male", "female"], size = (student_count))
student_id = list(range(0, student_count))
student_name_local = names_local
student_name_english = names_english
student_enrolled = random.choice(["Y", "N"], size = (student_count), p = [0.9, 0.1])
class_id = random.randint(0, classes_count, size = (student_count))



column_names = ['STUDENT_ID', 'STUDENT_NAME_LOCAL', 'STUDENT_NAME_ENGLISH', 'STUDENT_AGE', 'STUDENT_SEX', 'STUDENT_ENROLLED', 'STUDENT_FIRST_DAY', 'STUDENT_LAST_DAY']
students = pd.DataFrame(columns = column_names)

students['STUDENT_ID'] = student_id
students['STUDENT_NAME_LOCAL'] = student_name_local
students['STUDENT_NAME_ENGLISH'] = student_name_english
students['STUDENT_AGE'] = student_age
students['STUDENT_SEX'] = student_sex
students['CLASS_ID'] = class_id
students['STUDENT_ENROLLED'] = student_enrolled

## CLASSES.CSV

In [6]:
data = [['0', 'Wednesday', '16:30:00'], ['1', 'Wednesday', '17:30:00'], ['2', 'Wednesday', '18:30:00'], ['3', 'Thursday', '16:30:00'], ['4', 'Thursday', '17:30:00'], ['5', 'Thursday', '18:30:00'], ['6', 'Friday', '16:30:00'], ['7', 'Friday', '17:30:00'], ['8', 'Friday', '18:30:00'], ['9', 'Saturday', '8:30:00'], ['10', 'Saturday', '9:30:00'], ['11', 'Saturday', '10:30:00'], ['12', 'Saturday', '11:30:00'], ['13', 'Saturday', '13:30:00'], ['14', 'Saturday', '14:30:00'], ['15', 'Saturday', '15:30:00'], ['16', 'Saturday', '16:30:00'], ['17', 'Saturday', '17:30:00'], ['18', 'Sunday', '8:30:00'], ['19', 'Sunday', '9:30:00'], ['20', 'Sunday', '10:30:00'], ['21', 'Sunday', '11:30:00'], ['22', 'Sunday', '13:30:00'], ['23', 'Sunday', '14:30:00'], ['24', 'Sunday', '15:30:00'], ['25', 'Sunday', '16:30:00'], ['26', 'Sunday', '17:30:00']]

column_names = ['CLASS_ID', 'CLASS_DAY', 'CLASS_TIME']
classes = pd.DataFrame(data = data, columns = column_names)
classes['CLASS_ID'] = classes['CLASS_ID'].astype(int)

## ATTENDANCE.CSV

In [7]:
attendance_status = random.choice(["present", "absent"], p = [0.95, 0.05], size = workweeks_int)
attendance_list = []
a = {}
for i in student_id:
    ids = list([i])*workweeks_int
    column_names = ['STUDENT_ID', 'CLASS_DATE', 'ATTENDANCE_STATUS']
    a[i] = pd.DataFrame(columns = column_names)

    a[i]['STUDENT_ID'] = ids
    a[i]['ATTENDANCE_STATUS'] = attendance_status

    x = students['CLASS_ID'].loc[students['STUDENT_ID'] == i].to_list()
    x = x[-1]
    c = classes['CLASS_DAY'].loc[classes['CLASS_ID'] == x].to_list()
    c = c[-1]

    if c == 'Wednesday':
        a[i]['CLASS_DATE'] = wed_classes
    elif c == 'Thursday':
        a[i]['CLASS_DATE'] = thu_classes
    elif c == 'Friday':
        a[i]['CLASS_DATE'] = fri_classes[:-1]
    elif c == 'Saturday':
        a[i]['CLASS_DATE'] = sat_classes
    elif c == 'Sunday':
        a[i]['CLASS_DATE'] = sun_classes
    
    attendance_list.append(a[i])
attendance = pd.concat(attendance_list, ignore_index = True)

## TESTS.CSV

In [8]:
# WEDNESDAY CLASSES' TEST DAYS
wedDF = pd.DataFrame(data = wed_classes, columns = ['date'])
wedDF.index = wedDF['date']
wedDFg = wedDF.groupby(pd.Grouper(freq='M'))
wed_tests = wedDFg.agg({'date': np.max})['date'].tolist()
# THURSDAY CLASSES' TEST DAYS
thuDF = pd.DataFrame(data = thu_classes, columns = ['date'])
thuDF.index = thuDF['date']
thuDFg = thuDF.groupby(pd.Grouper(freq='M'))
thu_tests = thuDFg.agg({'date': np.max})['date'].tolist()
# FRIDAY CLASSES' TEST DAYS
friDF = pd.DataFrame(data = fri_classes, columns = ['date'])
friDF.index = friDF['date']
friDFg = friDF.groupby(pd.Grouper(freq='M'))
fri_tests = friDFg.agg({'date': np.max})['date'].tolist()
# SATURDAY CLASSES' TEST DAYS
satDF = pd.DataFrame(data = sat_classes, columns = ['date'])
satDF.index = satDF['date']
satDFg = satDF.groupby(pd.Grouper(freq='M'))
sat_tests = satDFg.agg({'date': np.max})['date'].tolist()
# SUNDAY CLASSES' TEST DAYS
sunDF = pd.DataFrame(data = sun_classes, columns = ['date'])
sunDF.index = sunDF['date']
sunDFg = sunDF.groupby(pd.Grouper(freq='M'))
sun_tests = sunDFg.agg({'date': np.max})['date'].tolist()

test_date = wed_tests + thu_tests + fri_tests + sat_tests + sun_tests
# DEFINE THE NUMBER OF TESTS
test_count = len(test_date)

tests_id = list(range(0, test_count))
tests_quantity = workmonths_int

column_names = ['TEST_ID', 'TEST_DATE']
tests = pd.DataFrame(columns = column_names)

tests['TEST_ID'] = tests_id
tests['TEST_DATE'] = test_date

# DEFINE THE NUMBER OF TESTS
test_count = len(test_date)

## TEST_RESULTS.CSV

### Function-ify It

### For Loop-ify It

In [9]:
TRlist = []
column_names = ['STUDENT_ID', 'TEST_SCORE', 'TEST_SCORE_MAX', 'TEST_SCORE_PERCENTAGE', 'TEST_DATE', 'CLASS_ID']
# Generate Test Result DataFrames for each Wednesday Class

wedTR = {}
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Wednesday'].to_list()
wed = []
for i in classIDs:
    class_id = i
    student_ids = students['STUDENT_ID'].loc[students['CLASS_ID'] == class_id].to_list()
    student_ids*len(wed_tests)
    testDates = list(np.repeat(wed_tests, len(student_ids)))
    test_score = random.randint(0, test_score_max, size = (len(testDates)))
    wedTR[i] = pd.DataFrame(columns = column_names)
    wedTR[i]['STUDENT_ID'] = student_ids*len(wed_tests)
    wedTR[i]['TEST_SCORE'] = test_score
    wedTR[i]['TEST_SCORE_MAX'] = test_score_max
    wedTR[i]['TEST_SCORE_PERCENTAGE'] = wedTR[i]['TEST_SCORE'] / wedTR[i]['TEST_SCORE_MAX']
    wedTR[i]['TEST_DATE'] = testDates
    wedTR[i]['CLASS_ID'] = class_id
    # Join this DataFrame with the Tests DataFrame on TEST_DATE
    wedTR[i] = pd.merge(wedTR[i], tests[['TEST_ID', 'TEST_DATE']], on = 'TEST_DATE', how = 'left')
    # Drop and save TEST_ID as an object
    testIDs = wedTR[i].pop('TEST_ID')
    # Insert TEST_ID into the first column position
    wedTR[i].insert(0, 'TEST_ID', testIDs)
    wed.append(wedTR[i])
wedTRDF = pd.concat(wed)
TRlist.append(wedTRDF)

# Generate Test Result DataFrames for each Thursday Class

thuTR = {}
thu = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Thursday'].to_list()
for i in classIDs:
    class_id = i
    student_ids = students['STUDENT_ID'].loc[students['CLASS_ID'] == class_id].to_list()
    student_ids*len(thu_tests)
    testDates = list(np.repeat(thu_tests, len(student_ids)))
    test_score = random.randint(0, test_score_max, size = (len(testDates)))
    thuTR[i] = pd.DataFrame(columns = column_names)
    thuTR[i]['STUDENT_ID'] = student_ids*len(thu_tests)
    thuTR[i]['TEST_SCORE'] = test_score
    thuTR[i]['TEST_SCORE_MAX'] = test_score_max
    thuTR[i]['TEST_SCORE_PERCENTAGE'] = thuTR[i]['TEST_SCORE'] / thuTR[i]['TEST_SCORE_MAX']
    thuTR[i]['TEST_DATE'] = testDates
    thuTR[i]['CLASS_ID'] = class_id
    # Join this DataFrame with the Tests DataFrame on TEST_DATE
    thuTR[i] = pd.merge(thuTR[i], tests[['TEST_ID', 'TEST_DATE']], on = 'TEST_DATE', how = 'left')
    # Drop and save TEST_ID as an object
    testIDs = thuTR[i].pop('TEST_ID')
    # Insert TEST_ID into the first column position
    thuTR[i].insert(0, 'TEST_ID', testIDs)
    thu.append(thuTR[i])
thuTRDF = pd.concat(thu)
TRlist.append(thuTRDF)

# Generate Test Result DataFrames for each Friday Class

friTR = {}
fri = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Friday'].to_list()
for i in classIDs:
    class_id = i
    student_ids = students['STUDENT_ID'].loc[students['CLASS_ID'] == class_id].to_list()
    student_ids*len(fri_tests)
    testDates = list(np.repeat(fri_tests, len(student_ids)))
    test_score = random.randint(0, test_score_max, size = (len(testDates)))
    friTR[i] = pd.DataFrame(columns = column_names)
    friTR[i]['STUDENT_ID'] = student_ids*len(fri_tests)
    friTR[i]['TEST_SCORE'] = test_score
    friTR[i]['TEST_SCORE_MAX'] = test_score_max
    friTR[i]['TEST_SCORE_PERCENTAGE'] = friTR[i]['TEST_SCORE'] / friTR[i]['TEST_SCORE_MAX']
    friTR[i]['TEST_DATE'] = testDates
    friTR[i]['CLASS_ID'] = class_id
    # Join this DataFrame with the Tests DataFrame on TEST_DATE
    friTR[i] = pd.merge(friTR[i], tests[['TEST_ID', 'TEST_DATE']], on = 'TEST_DATE', how = 'left')
    # Drop and save TEST_ID as an object
    testIDs = friTR[i].pop('TEST_ID')
    # Insert TEST_ID into the first column position
    friTR[i].insert(0, 'TEST_ID', testIDs)
    fri.append(friTR[i])
friTRDF = pd.concat(fri)
TRlist.append(friTRDF)

# Generate Test Result DataFrames for each Saturday Class

satTR = {}
sat = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Saturday'].to_list()
for i in classIDs:
    class_id = i
    student_ids = students['STUDENT_ID'].loc[students['CLASS_ID'] == class_id].to_list()
    student_ids*len(sat_tests)
    testDates = list(np.repeat(sat_tests, len(student_ids)))
    test_score = random.randint(0, test_score_max, size = (len(testDates)))
    satTR[i] = pd.DataFrame(columns = column_names)
    satTR[i]['STUDENT_ID'] = student_ids*len(sat_tests)
    satTR[i]['TEST_SCORE'] = test_score
    satTR[i]['TEST_SCORE_MAX'] = test_score_max
    satTR[i]['TEST_SCORE_PERCENTAGE'] = satTR[i]['TEST_SCORE'] / satTR[i]['TEST_SCORE_MAX']
    satTR[i]['TEST_DATE'] = testDates
    satTR[i]['CLASS_ID'] = class_id
    # Join this DataFrame with the Tests DataFrame on TEST_DATE
    satTR[i] = pd.merge(satTR[i], tests[['TEST_ID', 'TEST_DATE']], on = 'TEST_DATE', how = 'left')
    # Drop and save TEST_ID as an object
    testIDs = satTR[i].pop('TEST_ID')
    # Insert TEST_ID into the first column position
    satTR[i].insert(0, 'TEST_ID', testIDs)
    sat.append(satTR[i])
satTRDF = pd.concat(sat)
TRlist.append(satTRDF)

# Generate Test Result DataFrames for each Sunday Class

sunTR = {}
sun = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Sunday'].to_list()
for i in classIDs:
    class_id = i
    student_ids = students['STUDENT_ID'].loc[students['CLASS_ID'] == class_id].to_list()
    student_ids*len(sun_tests)
    testDates = list(np.repeat(sun_tests, len(student_ids)))
    test_score = random.randint(0, test_score_max, size = (len(testDates)))
    sunTR[i] = pd.DataFrame(columns = column_names)
    sunTR[i]['STUDENT_ID'] = student_ids*len(sun_tests)
    sunTR[i]['TEST_SCORE'] = test_score
    sunTR[i]['TEST_SCORE_MAX'] = test_score_max
    sunTR[i]['TEST_SCORE_PERCENTAGE'] = sunTR[i]['TEST_SCORE'] / sunTR[i]['TEST_SCORE_MAX']
    sunTR[i]['TEST_DATE'] = testDates
    sunTR[i]['CLASS_ID'] = class_id
    # Join this DataFrame with the Tests DataFrame on TEST_DATE
    sunTR[i] = pd.merge(sunTR[i], tests[['TEST_ID', 'TEST_DATE']], on = 'TEST_DATE', how = 'left')
    # Drop and save TEST_ID as an object
    testIDs = sunTR[i].pop('TEST_ID')
    # Insert TEST_ID into the first column position
    sunTR[i].insert(0, 'TEST_ID', testIDs)
    sun.append(sunTR[i])
sunTRDF = pd.concat(sun)
TRlist.append(sunTRDF)

test_results = pd.concat(TRlist)

## HOMEWORK_ASSIGNMENTS.CSV

### Function-ify It

In [10]:
def generateHomeworkAssignments(day):
    # Define global variables
    column_names = ['HW_ASSIGNED_DATE', 'HW_DUE_DATE', 'HW_SCORE_MAX', 'CLASS_ID']
    # Define an empty dictionary to store each class' DataFrame in
    HA = {}
    # Define an empty dictionary to store the day's DataFrame in
    hw = {}
    # Define an empty list to store the created DataFrames in for concatenating later
    HAList = {}
    # Define the Classes object
    if day == 'Wednesday':
        classes = wed_classes
    elif day == 'Thursday':
        classes = thu_classes
    elif day == 'Friday':
        classes = fri_classes
    elif day == 'Saturday':
        classes = sat_classes
    elif day == 'Sunday':
        classes = sun_classes
    # Get the Class IDs for the specified day
    classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == day].to_list()
    # Loop through the classes and create a DataFrame for each
    for i in classIDs:
        HA[i] = pd.DataFrame(columns = column_names)
        HA[i]['HW_ASSIGNED_DATE'] = classes
        HA[i]['CLASS_ID'] = i
        HA[i]['HW_DUE_DATE'] = HA[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
        HA[i]['HW_SCORE_MAX'].astype('Int64')
        HA[i]['HW_SCORE_MAX'] = hw_score_max
        # Append the DataFrame to the list
        HAList[day].append(HA[i])
    # Create a DataFrame from the above list
    hw[day] = pd.concat(HAList[day])
    # Return the DataFrame
    return hw[day]

In [92]:
# Define a list of Days
classDays = ['Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# Define the column names
column_names = ['HW_ASSIGNED_DATE', 'HW_DUE_DATE', 'HW_SCORE_MAX', 'CLASS_ID']
# Create an empty DataFrame to be the final DataFrame
hw_a = pd.DataFrame(columns = column_names)
# Create an empty dictionary to store the DataFrames in
homework = {}
# Loop through the days in the list of days to gerenerate a DataFrame for each
for i in classDays:
    homework[i] = generateHomeworkAssignments(i)
    # Append the returned DataFrame to the final DataFrame
    hw_a = pd.append(hw[i])

TypeError: list indices must be integers or slices, not str

In [93]:
hw[Wednesday] = generateHomeworkAssignments('Wednesday')

TypeError: list indices must be integers or slices, not str

### For Loop-ify It

In [11]:
# Define global variables
column_names = ['HW_ASSIGNED_DATE', 'HW_DUE_DATE', 'HW_SCORE_MAX', 'CLASS_ID']
HAList = []

# Generate Homework Assignment DataFrames for each Wednesday Class

wedHA = {}
wed = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Wednesday'].to_list()
for i in classIDs:
    wedHA[i] = pd.DataFrame(columns = column_names)
    wedHA[i]['HW_ASSIGNED_DATE'] = wed_classes
    wedHA[i]['CLASS_ID'] = i
    wedHA[i]['HW_DUE_DATE'] = wedHA[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
    wedHA[i]['HW_SCORE_MAX'].astype('Int64')
    wedHA[i]['HW_SCORE_MAX'] = hw_score_max
    wed.append(wedHA[i])
wedHADF = pd.concat(wed)
HAList.append(wedHADF)

# Generate Homework Assignment DataFrames for each Thursday Class

thuHA = {}
thu = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Thursday'].to_list()
for i in classIDs:
    thuHA[i] = pd.DataFrame(columns = column_names)
    thuHA[i]['HW_ASSIGNED_DATE'] = thu_classes
    thuHA[i]['CLASS_ID'] = i
    thuHA[i]['HW_DUE_DATE'] = thuHA[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
    thuHA[i]['HW_SCORE_MAX'].astype('Int64')
    thuHA[i]['HW_SCORE_MAX'] = hw_score_max
    thu.append(thuHA[i])
thuHADF = pd.concat(thu)
HAList.append(thuHADF)

# Generate Homework Assignment DataFrames for each Friday Class

friHA = {}
fri = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Friday'].to_list()
for i in classIDs:
    friHA[i] = pd.DataFrame(columns = column_names)
    friHA[i]['HW_ASSIGNED_DATE'] = fri_classes
    friHA[i]['CLASS_ID'] = i
    friHA[i]['HW_DUE_DATE'] = friHA[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
    friHA[i]['HW_SCORE_MAX'].astype('Int64')
    friHA[i]['HW_SCORE_MAX'] = hw_score_max
    fri.append(friHA[i])
friHADF = pd.concat(fri)
HAList.append(friHADF)

# Generate Homework Assignment DataFrames for each Saturday Class

satHA = {}
sat = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Saturday'].to_list()
for i in classIDs:
    satHA[i] = pd.DataFrame(columns = column_names)
    satHA[i]['HW_ASSIGNED_DATE'] = sat_classes
    satHA[i]['CLASS_ID'] = i
    satHA[i]['HW_DUE_DATE'] = satHA[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
    satHA[i]['HW_SCORE_MAX'].astype('Int64')
    satHA[i]['HW_SCORE_MAX'] = hw_score_max
    sat.append(satHA[i])
satHADF = pd.concat(sat)
HAList.append(satHADF)

# Generate Homework Assignment DataFrames for each Sunday Class

sunHA = {}
sun = []
classIDs = classes['CLASS_ID'].loc[classes['CLASS_DAY'] == 'Sunday'].to_list()
for i in classIDs:
    sunHA[i] = pd.DataFrame(columns = column_names)
    sunHA[i]['HW_ASSIGNED_DATE'] = sun_classes
    sunHA[i]['CLASS_ID'] = i
    sunHA[i]['HW_DUE_DATE'] = sunHA[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
    sunHA[i]['HW_SCORE_MAX'].astype('Int64')
    sunHA[i]['HW_SCORE_MAX'] = hw_score_max
    sun.append(sunHA[i])
sunHADF = pd.concat(sun)
HAList.append(sunHADF)

hw_a = pd.concat(HAList)
hw_a['HW_ID'] = list(range(0, hw_a.shape[0]))

## HOMEWORK_GRADES.CSV

### For Loop-ify It

In [15]:
# SET GLOBAL VARS
column_names = ['STUDENT_ID', 'HW_ID', 'HW_SCORE', 'HW_SCORE_MAX']
hw_c = {}
hw_g_list = []
# Get a list of Class IDs
class_ids = classes['CLASS_ID'].to_list()
# Loop through the Class IDs and create a DataFrame of scores for each of them
for i in class_ids:
    student_id = students['STUDENT_ID'].loc[students['CLASS_ID'] == i]
    hw_id = hw_a['HW_ID'].loc[hw_a['CLASS_ID'] == i]
    hw_score = random.randint(0, 10, size = (len(student_id)*len(hw_id)))
    hw_c[i] = pd.DataFrame(columns = column_names)
    hw_c[i]['STUDENT_ID'] = list(student_id)*len(hw_id)
    hw_c[i]['HW_ID'] = list(hw_id)*len(student_id)
    hw_c[i]['HW_SCORE'] = hw_score
    hw_c[i]['HW_SCORE_MAX'] = hw_score_max
    hw_c[i]['HW_SCORE_PERCENT'] = hw_score / hw_c[i]['HW_SCORE_MAX']
    hw_c[i] = pd.merge(hw_c[i], hw_a[['HW_ID', 'HW_ASSIGNED_DATE']], on = 'HW_ID', how = 'left')
    hw_c[i]['HW_TURNED_IN_DATE'] = hw_c[i]['HW_ASSIGNED_DATE'] + timedelta(days = 7)
    hw_c[i]['CLASS_ID'] = i
    hw_g_list.append(hw_c[i])

hw_g = pd.concat(hw_g_list)

In [16]:
hw_g

,STUDENT_ID,HW_ID,HW_SCORE,HW_SCORE_MAX,HW_SCORE_PERCENT,HW_ASSIGNED_DATE,HW_TURNED_IN_DATE,CLASS_ID
0,26,0,9,10,0.9,2013-03-06,2013-03-13,0
1,31,1,2,10,0.2,2013-03-13,2013-03-20,0
2,120,2,4,10,0.4,2013-03-20,2013-03-27,0
3,132,3,0,10,0.0,2013-03-27,2013-04-03,0
4,137,4,1,10,0.1,2013-04-03,2013-04-10,0
...,...,...,...,...,...,...,...,...
1885,351,3400,3,10,0.3,2015-06-28,2015-07-05,26
1886,382,3401,8,10,0.8,2015-07-05,2015-07-12,26
1887,406,3402,4,10,0.4,2015-07-12,2015-07-19,26
1888,446,3403,0,10,0.0,2015-07-19,2015-07-26,26


## PARTICIPATION.CSV

In [13]:
student_id = random.randint(student_count, size = (participation_count))
participation_type = random.choice(["volunteer", "cold call"], size = (participation_count))
participation_attempts = random.randint(1, 6, size = (participation_count))
participation_hints = random.randint(0, participation_attempts, size = (participation_count))
    
column_names = ['STUDENT_ID', 'PARTICIPATION_DATETIME', 'PARTICIPATION_TYPE', 'PARTICIPATION_ATTEMPTS', 'PARTICIPATION_HINTS']
participation = pd.DataFrame(columns = column_names)

participation['STUDENT_ID'] = student_id
participation['PARTICIPATION_TYPE'] = participation_type
participation['PARTICIPATION_ATTEMPTS'] = participation_attempts
participation['PARTICIPATION_HINTS'] = participation_hints

participation = pd.merge(participation, students[['STUDENT_ID', 'CLASS_ID']], on = 'STUDENT_ID', how = 'left')
participation = pd.merge(participation, classes[['CLASS_ID', 'CLASS_TIME']], on = 'CLASS_ID', how = 'left')
participation = pd.merge(participation, classes[['CLASS_ID', 'CLASS_DAY']], on = 'CLASS_ID', how = 'left')

dayList = participation['CLASS_DAY'].to_list()

# Insert random dates for each participation
partDate = []
for row in dayList:
    if row == "Wednesday":
         partDate.append(random.choice(wed_classes))
    elif row == "Thursday":
         partDate.append(random.choice(thu_classes))
    elif row == "Friday":
         partDate.append(random.choice(fri_classes))
    elif row == "Saturday":
         partDate.append(random.choice(sat_classes))
    elif row == "Sunday":
         partDate.append(random.choice(sun_classes))

participation['PARTICIPATION_DATETIME'] = partDate

# Insert a random timestamp for each participation
    # Leave out for now as I can't think of a way to be able to implement it on the website

# EXPORT THE DATAFRAMES

## LAPPIE

In [ ]:
# Export the pandas dataframes as CSVs 
students.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\students.csv', index = False, header = True)
classes.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\classes.csv', index = False, header = True)
attendance.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\attendance.csv', index = False, header = True)
tests.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\tests.csv', index = False, header = True)
hw_g.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\hw_grades.csv', index = False, header = True)
hw_a.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\hw_assignments.csv', index = False, header = True)
participation.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\participation.csv', index = False, header = True)
tests_results.to_csv(r'C:\Users\mfitz\Google Drive\Docs\VSCode\Students Data\China\Tables\test_results.csv', index = False, header = True)

## GLaDOS

In [17]:
# Export the pandas dataframes as CSVs 
students.to_csv(r'Tables\students.csv', index = False, header = True)
classes.to_csv(r'Tables\classes.csv', index = False, header = True)
attendance.to_csv(r'Tables\attendance.csv', index = False, header = True)
tests.to_csv(r'Tables\tests.csv', index = False, header = True)
hw_g.to_csv(r'Tables\hw_grades.csv', index = False, header = True)
hw_a.to_csv(r'Tables\hw_assignments.csv', index = False, header = True)
participation.to_csv(r'Tables\participation.csv', index = False, header = True)
test_results.to_csv(r'Tables\test_results.csv', index = False, header = True)